In [9]:
from diffusers import AutoPipelineForText2Image
import torch
import inspect

pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16")

# __call__ 메서드의 파라미터 확인
params = inspect.signature(pipe.__call__).parameters
for param_name, param in params.items():
    print(f"{param_name}: {param}")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

prompt: prompt: Union[str, List[str]] = None
prompt_2: prompt_2: Union[str, List[str], NoneType] = None
height: height: Union[int, NoneType] = None
width: width: Union[int, NoneType] = None
num_inference_steps: num_inference_steps: int = 50
timesteps: timesteps: List[int] = None
denoising_end: denoising_end: Union[float, NoneType] = None
guidance_scale: guidance_scale: float = 5.0
negative_prompt: negative_prompt: Union[str, List[str], NoneType] = None
negative_prompt_2: negative_prompt_2: Union[str, List[str], NoneType] = None
num_images_per_prompt: num_images_per_prompt: Union[int, NoneType] = 1
eta: eta: float = 0.0
generator: generator: Union[torch._C.Generator, List[torch._C.Generator], NoneType] = None
latents: latents: Union[torch.FloatTensor, NoneType] = None
prompt_embeds: prompt_embeds: Union[torch.FloatTensor, NoneType] = None
negative_prompt_embeds: negative_prompt_embeds: Union[torch.FloatTensor, NoneType] = None
pooled_prompt_embeds: pooled_prompt_embeds: Union[torch.Floa

In [3]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 10.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
import spacy
import os
import pandas as pd
from pathlib import Path

# dataß
caption_dir = Path("/mnt/nas65/Dataset/ImageNet1K/ILSVRC/Data/CLS-LOC/train_csv")
cls_mapping = pd.read_csv("./map_cls.txt", sep=' ', header=None, names=['cls_name','cls_num', 'cls_id'])
# model
nlp = spacy.load("en_core_web_sm")


In [ ]:
from gensim.models import KeyedVectors

# 사전학습된 모델 로드 (예: Google News dataset의 Word2Vec 모델)
model_path = 'GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(model_path, binary=True)

# 캡션과 비교할 단어
target_word = 'cat'

# 캡션 내의 단어들 (예시)
caption_words = ['dog', 'mouse', 'cat', 'pet']

# 각 단어와 타겟 단어 사이의 유사도 계산
similarities = {word: model.similarity(target_word, word) for word in caption_words if word in model.key_to_index}

# 가장 유사도가 높은 단어 찾기
most_similar_word = max(similarities, key=similarities.get)

print("가장 유사한 단어:", most_similar_word)


In [ ]:
import pandas as pd
from pathlib import Path

# 캡션 데이터 로드
path = Path('/mnt/nas65/Dataset/ImageNet1K/ILSVRC/Data/CLS-LOC/train_csv/n01440764.csv')
csvfile = pd.read_csv(path)

In [ ]:
csvfile['caption'][0]

In [ ]:
cls_mapping = Path('./map_cls.txt')
cls_mapping = pd.read_csv(cls_mapping, sep=' ',index_col='cls_name', header=None, names=['cls_name','cls_num', 'cls_id'])
cls_mapping

In [3]:
from transformers import BertTokenizer, BertModel
import torch

# 모델과 토크나이저 초기화
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Working on {device}")
model.eval()

# 문장과 타겟 단어 설정
sentence = "I enjoy watching football and basketball."
target_word = "soccer"

# 문장을 토큰화하고 BERT 입력 형식으로 변환
tokens = tokenizer.tokenize(sentence)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
tokens_tensor = torch.tensor([indexed_tokens]).to(device)

# 모델을 통해 임베딩 계산
with torch.no_grad():
    outputs = model(tokens_tensor)
    hidden_states = outputs.last_hidden_state.squeeze()

# 타겟 단어의 임베딩을 계산
target_tokens = tokenizer.tokenize(target_word)
target_indexed_tokens = tokenizer.convert_tokens_to_ids(target_tokens)
target_tokens_tensor = torch.tensor([target_indexed_tokens]).to(device)
with torch.no_grad():
    target_outputs = model(target_tokens_tensor)
    target_hidden_states = target_outputs.last_hidden_state.squeeze()
    target_hidden_state = target_hidden_states.mean(dim=0)

# 유사도 계산 (코사인 유사성)
cosine_similarities = torch.nn.functional.cosine_similarity(hidden_states, target_hidden_state.unsqueeze(0), dim=1)

# 가장 유사한 단어 찾기
most_similar_token_idx = torch.argmax(cosine_similarities).item()
most_similar_word = tokens[most_similar_token_idx]

# 원래 문장에서 단어 대치
new_sentence = sentence.replace(most_similar_word, target_word)
print("Original sentence:", sentence)
print("Modified sentence:", new_sentence)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Working on cuda
Original sentence: I enjoy watching football and basketball.
Modified sentence: I enjoy watching football and basketballsoccer


In [1]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import os

import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag


class CSVModifier:
    def __init__(self, model_name='bert-base-uncased', caption_dir="/mnt/nas65/Dataset/ImageNet1K/ILSVRC/Data/CLS-LOC/train_csv", save_dir="/path/to/save"):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name, output_hidden_states=True)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.model.eval()

        self.caption_dir = caption_dir
        self.save_dir = save_dir
        self.mapping = pd.read_csv("/home/tak/IBT/Image-back-translation/notebooks/submaterial/map_cls.txt", sep=' ', header=None, names=['cls_name', 'cls_num', 'cls_id'])
        self.mapping_dict = {key: val.replace("_"," ") for key, val in zip(self.mapping['cls_name'], self.mapping['cls_id'])}

    def modify_all_csvs(self):
        for csv_file in os.listdir(self.caption_dir):
            if csv_file.endswith('.csv'):
                self.modify_csv(csv_file)

    # def modify_csv(self, csv_file):
    #     df = pd.read_csv(os.path.join(self.caption_dir, csv_file), sep=r",(?:(?!\s)+(?!')+(?!$))", engine='python')
    #     df['modified_caption'] = self.replace_words(df['caption'].tolist(), csv_file.replace('.csv', ''))

    #     df.to_csv(os.path.join(self.save_dir, f"{csv_file.replace('.csv', '')}_modified.csv"), index=False)

    def modify_csv(self, csv_file):
        df = pd.read_csv(os.path.join(self.caption_dir, csv_file), sep=r",(?:(?!\s)+(?!')+(?!$))", engine='python')
        df['modified_caption'] = self.replace_words(df['caption'].tolist(), csv_file.replace('.csv', ''))

        # Save the DataFrame as a JSON file
        json_file_path = os.path.join(self.save_dir, f"{csv_file.replace('.csv', '')}_modified.json")
        df.to_json(json_file_path, orient='records', lines=True)


    # def replace_words(self, texts, class_folder):
    #     class_embedding = self.get_class_embedding(class_folder)
    #     tokenized = self.tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    #     tokenized.to(self.device)
    #     with torch.no_grad():
    #         outputs = self.model(**tokenized)
    #         hidden_states = outputs.last_hidden_state

    #     modified_texts = []
    #     class_name = self.mapping_dict.get(class_folder, class_folder)  # 클래스 폴더명에 해당하는 클래스 이름 가져오기
    #     for idx, text in enumerate(texts):
    #         hidden_state = hidden_states[idx]
    #         similarities = torch.nn.functional.cosine_similarity(hidden_state, class_embedding.unsqueeze(0), dim=1)
    #         most_similar_token_idx = torch.argmax(similarities).item()

    #         most_similar_word_id = tokenized['input_ids'][idx][most_similar_token_idx].unsqueeze(0)
    #         most_similar_word = self.tokenizer.convert_ids_to_tokens(most_similar_word_id)[0]

    #         # 가장 유사한 단어를 클래스 이름으로 대체
    #         text = text.replace(most_similar_word, class_name)
    #         modified_texts.append(text)

    #     return modified_texts
    
    def replace_words(self, texts, class_folder):
        class_embedding = self.get_class_embedding(class_folder)
        tokenized = self.tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
        tokenized.to(self.device)
        with torch.no_grad():
            outputs = self.model(**tokenized)
            hidden_states = outputs.last_hidden_state

        modified_texts = []
        class_name = self.mapping_dict.get(class_folder, class_folder)
        for idx, text in enumerate(texts):
            # POS tagging to filter nouns
            words = word_tokenize(text)
            pos_tags = pos_tag(words)
            nouns = [word for word, tag in pos_tags if tag.startswith('NN')]

            hidden_state = hidden_states[idx]
            similarities = torch.nn.functional.cosine_similarity(hidden_state, class_embedding.unsqueeze(0), dim=1)
            token_ids = tokenized['input_ids'][idx]
            token_texts = self.tokenizer.convert_ids_to_tokens(token_ids)

            # Filter similarities for nouns only
            noun_indices = [i for i, token in enumerate(token_texts) if token in nouns]
            if not noun_indices:
                modified_texts.append(text)
                continue

            noun_similarities = similarities[noun_indices]
            most_similar_noun_idx = torch.argmax(noun_similarities).item()
            most_similar_noun = token_texts[noun_indices[most_similar_noun_idx]]

            # Replace the most similar noun with the class name
            text = text.replace(most_similar_noun, class_name)
            modified_texts.append(text)

        return modified_texts



    def get_class_embedding(self, class_folder):
        class_name = self.mapping_dict.get(class_folder, class_folder)  # Assume class_folder is a valid key
        tokens_tensor = torch.tensor([self.tokenizer.convert_tokens_to_ids(self.tokenizer.tokenize(class_name))]).to(self.device)
        with torch.no_grad():
            outputs = self.model(tokens_tensor)
            class_embedding = outputs.last_hidden_state.squeeze().mean(dim=0)
        return class_embedding

# Usage example
modifier = CSVModifier(save_dir="/home/tak/IBT/Image-back-translation/notebooks/submaterial/aligned_json")

/home/tak/miniconda3/envs/lavis/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tak/miniconda3/envs/lavis/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
modifier.modify_all_csvs()